In [ ]:
from lecture import *

# Introduction to programming in Python
### [Gerard Gorman](http://www.imperial.ac.uk/people/g.gorman)

# Lecture 8: Introduction to classes

Learning objectives: 

* Learn how to create your own **objects** in Python and develop **member functions** for these new data types.

## Class: encapsulating variables/data and functions

A class encapsulates variables/data and functions into one single unit. As a programmer you can create a new class and thereby a new **object type** (similar to those you have already encountered - int, `float`, `string`, `list`, `file`, etc.). Once you have created a class you can create many instances of that type as you wish, just as you can have many `int` or `float` objects.

Modern programming makes heavy use of classes and object orientated programming to manage software complexity, making these important concepts to understand. However, for non-trivial applications the design of good abstractions and classes requires careful consideration, otherwise one can unintentionally increase complexity and hurt the performance of your code. Therefore, you should consider this lecture merely as a gentle introduction illustrated with some simple examples.

## Representing a function by a class

Consider a function of $t$ with a parameter $v_0$:
$$ y(t: v_0, g)=v_0t - {1\over2}gt^2 $$

We need both $v_0$, $g$ and $t$ to evaluate $y$. How might we implement this?

One option is to assume we will always pass in all variables as arguments:
```python
def y(t, v0, g=9.81):
    return v0*t - 0.5*g*t**2
```
This looks like a reasonable solution when there are only a couple of parameters. But the software complexity quickly gets out of hand as the number of variables increases (I have worked on legacy codes that had function argument lists that were hundreds of lines long because there was no notion of encapsulation!)

Alternatively we might define `v0` and `g` as global variables:
```python
g=9.81
v0 = ...

...

def y(t):
    return v0*t - 0.5*g*t**2
```
However, the use of global variables is strongly discouraged for many reasons, e.g. very error prone, increased risk of namespace pollution (variables being clobbered when you import a Python module), makes it difficult to manage instances where there might be multiple values for the global variable within the same context, etc.

Lets look at how we might instead implement this as a class.

While we will not cover it in detail here, it is worth noting that professional developers often use [UML (Unified Modeling Language)](http://en.wikipedia.org/wiki/Unified_Modeling_Language) to illustrate the design of a class. Here is a UML diagram for this example:

![Simple UML example](https://github.com/ggorman/Introduction-to-programming-for-geoscientists/raw/master/notebook/images/class_Y_UML.png)

For this example `class Y` for $y(t: v_0, g)$ has variables `v0` and `g` and a function `y(t)` for computing $y(t: v_0, g)$. Often classes also have the special function `__init__` for initialising class variables.

Here is an implementation of this class:

In [ ]:
class Y:
    def __init__(self, v0, g=9.81):
        self.v0 = v0
        self.g = g
        
    def value(self, t):
        return self.v0*t - 0.5*self.g*t**2

An example of its usage: 

In [ ]:
y = Y(v0=3)      # Create instance
v = y.value(0.1) # Compute function value

print(v)

When we write `y = Y(v0=3)` we create a new *instance* of *type ` Y`*.

`Y(3)` is a call to the constructor:

```python
def __init__(self, v0, g=9.81):
    self.v0 = v0
    self.g = g
```

Think of `self` as `y`, *i.e.*, the new variable to be created. `self.v0` means that we attach a variable `v0` to self (`y`).

```python
Y.__init__(y, 3)   # is the logic behind Y(3)
```

`self` is always the first argument/parameter in a function, but **never** inserted in the call! After `y=Y(3)`, `y` has two variables `v0` and `g`, and we can take a look at these:

In [ ]:
print(y.v0)
print(y.g)

Functions in classes are called **methods**. Variables in classes are called **attributes**. Therefore, in the above example the `value` *method* was

```python
def value(self, t):
  return self.v0*t - 0.5*self.g*t**2
```

Example on a call:

In [ ]:
v = y.value(t=0.1)

`self` is left out in the call (as discussed above), but Python automatically inserts `y` as the `self` argument inside the `value` method. Inside the `value` *method* things *appear* as

```python
return y.v0*t - 0.5*y.g*t**2
```

The method `value` has, through `self`, access to the attributes. Attributes are like *global variables* in the class, and any method gets a `self` parameter as its first argument. The method can then access the attributes of the class through `self`.

In summary, `class Y` collects the attributes `v0` and `g` and the method `value` together as a single unit. `value(t)` is function of `t` only, but has access to the class attributes `v0` and `g`.

The great feature of Python is that we can send `y.value` as an ordinary function of `t` to any other function that expects a function `f(t)`:

In [ ]:
import numpy as np

def table(f, tstop, n):
    """Make a table of t, f(t) values."""
    for t in np.linspace(0, tstop, n):
        print(t, f(t))

In [ ]:
def g(t):
    return np.sin(t)*np.exp(-t)

table(g, 2*np.pi, 5) # pass in ordinary function as first argument

In [ ]:
y = Y(6.5)
table(y.value, 2*np.pi, 5) # pass in class method as first argument

## Exercise 8.1: Make a class for function evaluation.
Make a class called *F* that implements the function

$$f(x: a, w) = \exp(−ax)\sin(wx).$$

A *value(x)* method computes values of *f* for a given `x`, while *a* and *w* are class attributes as specified as keyword arguments in the class `__init__` method.

In [ ]:
# Uncomment and complete this code - keep the names the same for testing purposes. 

# class F:
#     ...

In [ ]:
ok.grade('question-8_1')

## Exercise 8.2: Make a simple class
Make a class called *Simple* with:
* one attribute, `i`
* one method *double* that replaces the value of *i* by *i+i*
* an `__init__` method that initializes the attribute. 

Use the following code snippet to convince yourself that your class is behaving as expected.

```python
s1 = Simple(4)
for i in range(4):
    s1.double()
print(s1.i)

s2 = Simple('Hello')
s2.double(); s2.double()
print(s2.i)
s2.i = 100
print(s2.i)
```

In [ ]:
ok.grade('question-8_1')

## Another class example: a bank account

* Attributes: name of owner, account number, balance
* Methods: deposit, withdraw, pretty print

In [ ]:
class Account:
    
    def __init__(self, name, account_number, initial_amount=0):
        self.name = name
        self.no = account_number
        self.balance = initial_amount
        
    def deposit(self, amount):
        self.balance += amount
        
    def withdraw(self, amount):
        self.balance -= amount
        
    def dump(self):
        s = '%s, %s, balance: %s' % (self.name, self.no, self.balance)
        print(s)

In [ ]:
a1 = Account('John Olsson', '19371554951')
a2 = Account('Liz Olsson', '19371564761', 20000)
a1.deposit(1000)
a1.withdraw(4000)
a2.withdraw(10500)
a1.withdraw(3500)
print("%s’s balance: %.2f"%(a1.name, a1.balance))

In [ ]:
a1.dump()

In [ ]:
a2.dump()

## Exercise 8.3: Extend a class

Add an attribute called `transactions` to the `Account` class given above. The new attribute counts the number of transactions done in the `deposit` and `withdraw` methods. The total number of transactions should be printed in the `dump` method. Write a simple test program to convince yourself transaction gets the right value after some calls to `deposit` and `withdraw`.

In [ ]:
ok.grade('question-8_3')

## Protecting attributes
It is not possible in Python to explicitly protect attributes from being overwritten by the calling function, *i.e.* the following is possible but not intended:

In [ ]:
a1.name = 'Some other name'
a1.balance = 100000
a1.no = '19371564768'

**Assumptions** on correct usage include:

* The attributes should not be modified directly.
* The `balance` attribute can be viewed.
* Changing `balance` is done through with the methods `draw` and `deposit`.

The remedy is to adopt the convention that attributes and methods not intended for use outside the class should be marked as protected by prefixing the name with an underscore (*e.g.*, `_name`). This is just a convention to warn you to stay away from messing with the attribute directly. There is no technical way of stopping attributes and methods from being accessed directly from outside the class.

We rewrite the account class using this convention:

In [ ]:
class AccountP:
    def __init__(self, name, account_number, initial_amount):
        self._name = name
        self._no = account_number
        self._balance = initial_amount
    def deposit(self, amount):
        self._balance += amount
    def withdraw(self, amount):
        self._balance -= amount
    def get_balance(self):    # NEW - read balance value
        return self._balance
    def dump(self):
        s = '%s, %s, balance: %s' %(self._name, self._no, self._balance)
        print(s)

In [ ]:
a1 = AccountP('John Olsson', '19371554951', 20000)
a1.withdraw(4000)

In [ ]:
print(a1._balance)      # it works, but a convention is broken

In [ ]:
print(a1.get_balance()) # correct way of viewing the balance

In [ ]:
a1._no = '19371554955' # if you did this you'd probably lose your job! Don't mess with the convention.

### Example - a phone book

A phone book is a list of data about persons. Typical data includes: name, mobile phone, office phone, private phone, email. This data about a person can be  collected in a class as **attributes**. Think about what kinds of **methods** make sense for this class, e.g.:

* Constructor for initializing name, plus one or more other data
* Add new mobile number
* Add new office number
* Add new private number
* Add new email
* Write out person data

In [ ]:
class Person:
    def __init__(self, name, mobile_phone=None, office_phone=None, private_phone=None, email=None):
        self.name = name
        self.mobile = mobile_phone
        self.office = office_phone
        self.private = private_phone
        self.email = email
    def add_mobile_phone(self, number):
        self.mobile = number
    def add_office_phone(self, number):
        self.office = number
    def add_private_phone(self, number):
        self.private = number
    def add_email(self, address):
        self.email = address
    def dump(self):
        s = self.name + '\n'
        if self.mobile is not None:
            s += 'mobile phone:   %s\n' % self.mobile
        if self.office is not None:
            s += 'office phone:   %s\n' % self.office
        if self.private is not None:
            s += 'private phone:  %s\n' % self.private
        if self.email is not None:
            s += 'email address:  %s\n' % self.email
        print(s)

In [ ]:
p1 = Person('Gerard Gorman', email='g.gorman@imperial.ac.uk')
p1.add_office_phone('49985')

p2 = Person('ICT Service Desk', office_phone='49000')
p2.add_email('service.desk@imperial.ac.uk')

phone_book = {'Gorman': p1, 'ICT': p2}
for p in phone_book:
    phone_book[p].dump()

### Example - a circle
A circle is defined by its center point $x0, y0$ and its radius $R$. These data can be attributes in a class. Possible methods in the class are *area* and *circumference*. The constructor initializes $x0$, $y0$ and $R$.

In [ ]:
class Circle:
    def __init__(self, R, x0, y0,):
        self.x0, self.y0, self.R = x0, y0, R
    def area(self):
        return np.pi*self.R**2
    def circumference(self):
        return 2*np.pi*self.R

In [ ]:
c = Circle(2, -1, 5)
print('A circle with radius %g at (%g, %g) has area %g' % (c.R, c.x0, c.y0, c.area()))

## Exercise 8.4: Make a class for straight lines

Make a class called *Line* whose constructor takes two points `p0` and `p1` (2-tuples or 2-lists) as input. The line goes through these two points (see function *line* defined below for the relevant formula of the line). A *value(x)* method computes the `y` value on the line at the point *x* or returns `None` if the line is vertical (i.e. `(x1-x0)==0`). 

```python
def line(x0, y0, x1, y1):
    """
    Compute the coefficients a and b in the mathematical
    expression for a straight line y = a*x + b that goes
    through two points (x0, y0) and (x1, y1).
    x0, y0: a point on the line (floats).
    x1, y1: another point on the line (floats).
    return: coefficients a, b (floats) for the line (y=a*x+b).
    """
    try:
        a = (y1 - y0)/(x1 - x0)
        b = y0 - a*x0
    except ZeroDivisionError:
        a, b = None, None
    
    return a, b
```

In [ ]:
ok.grade('question-8_4')

## Exercise 8.5: Make a class for quadratic functions

Consider a quadratic function $f(x; a, b, c) = ax^2 + bx + c$. Make a class called *Quadratic* for representing *f*, where *a*, *b*, and *c* are attributes, and the methods are:

1. *value* for computing a value of *f* at a point *x*,
2. *table* for writing out a table of *x* and *f* values for n *x* values in the
interval *[L, R]*,
3. *roots* for computing the two roots.

In [ ]:
ok.grade('question-8_5')

## Special methods

Some class methods have leading and trailing double underscores. You have already met one of these, `__init__` used to initialise an object upon creation. Other examples include `__call__(self, ...)` and `__add__(self, other)`. These *special methods* enable more elegant abstractions and interfaces. Consider for example the difference between the equivalent statements:

```python
y = Y(4)
```
rather than
```python
y = Y
Y.__init__(Y, 4)```

### Special member function, `__call__`: make the class instance behave and look as a function

Let us replace the `value` method in `class Y` by a `__call__` special method:

In [ ]:
class Y:
    def __init__(self, v0, g=9.81):
        self.v0 = v0
        self.g = g
    
    def __call__(self, t):
        return self.v0*t - 0.5*self.g*t**2

Now we can write:

In [ ]:
y = Y(3)
v = y(0.1) # same as v = y.__call__(0.1)

The instance $y$ behaves/looks as a function! The `value(t)` method in the first example does the same, but the special method `__call__` provides a more elegant and concise syntax for computing function values.

### Special member function, `__str__`: represent object as a string for printing

In Python, we can usually print an object `a` by `print(a)`. This works for built-in types (strings, lists, floats, ...). However, if we have made a new type through a class, Python does not know how to print objects of this type. However, if the class has defined a method `__str__` , Python will use this method to convert the object to a string.

In [ ]:
class Y:
    def __init__(self, v0, g=9.81):
        self.v0 = v0
        self.g = g
    def __call__(self, t):
        return self.v0*t - 0.5*self.g*t**2
    def __str__(self):
        return '%g*t - 0.5*%g*t**2' % (self.v0, self.g)

In [ ]:
y = Y(1.5)

print(y)

### Special methods for overloading arithmetic operations

```python
c=a+b               # c = a.__add__(b)
c=a-b               # c = a.__sub__(b)
c = a*b             # c = a.__mul__(b) 
c = a/b             # c = a.__div__(b) 
c = a**e            # c = a.__pow__(e)```

### Special methods for overloading conditional operations

```python
a == b               #  a.__eq__(b)
a != b               #  a.__ne__(b)
a < b                #  a.__lt__(b)
a <= b               #  a.__le__(b)
a > b                #  a.__gt__(b)
a >= b               #  a.__ge__(b)```

In [ ]:
ok.score()